# Experiments with LangChain

This notebook contains several code snippets and experiments done with [LangChain](https://www.langchain.com/).

## Prompt for text classification

The first thing we're going to do is to implement the same prompt we did in `experiments` for text classification, but using LangChain.

In [1]:
from langchain.prompts import ChatPromptTemplate
import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    StoppingCriteria, 
    StoppingCriteriaList, 
    TextIteratorStreamer
    )
from samples import samples

/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
template_string = """
Instruct: Classify the following text in one of the following categories: ["support", "sales", "joke"]. Output only the name of the category.
+ "support" for customer support texts
+ "sales" for sales and comercial texts
+ "joke" for jokes, funny or comedy like texts
Text: {text}
Output:
""".strip()

In [3]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [4]:
print("Input variables are", prompt_template.messages[0].input_variables)

Input variables are ['text']


In [5]:
print(prompt_template.format_messages(text="My package was lost")[0].content)

Instruct: Classify the following text in one of the following categories: ["support", "sales", "joke"]. Output only the name of the category.
+ "support" for customer support texts
+ "sales" for sales and comercial texts
+ "joke" for jokes, funny or comedy like texts
Text: My package was lost
Output:


## Connect `langchain` with `phi-2` and `huggingface`

Ok, so now that we have a way to create prompts for our text classification using LangChain, we need to connect it with the AI model.

Let's start with some already know code to load and use the `phi-2` model.

In [6]:
class StopOnTokens(StoppingCriteria):
    """Stops the model if it produces an 'end of text' token"""
    def __call__(self, input_ids: torch.LongTensor, 
                 scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [50256, 198] # <|endoftext|> and EOL
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Your device is", device)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2", 
    device_map="auto", 
    torch_dtype="auto" if device == "cuda" else torch.float, 
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

Your device is cuda


Loading checkpoint shards: 100%|██████████████████████████████████████████████| 2/2 [00:00<00:00,  2.43it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


To do that, we need to do it through `langchain`'s `HuggingFacePipeline`.

In [7]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200, stopping_criteria=[StopOnTokens()])
hf = HuggingFacePipeline(pipeline=pipe)

Now define a new chain (with the `prompt_template` we defined in the previous section) and test it with some random sentence from our dataset. To do so, we use [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/), which is the new way of composing chains together.

In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | hf | StrOutputParser()

In [9]:
from random import choice

category = choice(list(samples.keys()))
text = choice(samples[category])

response = chain.invoke({"text": text})

print("PREDICTED:", response.strip())
print("TRUE CATEGORY", category)
print("TEXT:", text)

PREDICTED: joke
TRUE CATEGORY joke
TEXT: How does a chatbot make a decision? It uses algorithm-ination!


As we can see, our first `chain` is able to classify the input `text` in the following categories: support, sales and joke.

## Routing

The next thing we have to do is to implement some routing to branch depending on the classification result. We want the chatbot to accomodate to the tone and content of the conversation. Thus, we'll set different instructions to follow depending on whether we're talking about _customer support_, _sales_, or simply _joking_.

We're going to use `RunnableBranch` to run a different chain depending on the classification output. In our case we're going to implement a different prompt each time. There is also a generic prompt in the case the classifier is unable to undertand the content of the text or the output of the model is not properly parsed.

Below, we define four different prompts and chains.

In [10]:
from langchain.prompts import PromptTemplate

support_template = """\
Instruction: You are a customer support agent. It seems that the user may have some issues. Answer to their query politely and sincerely. \
Be kind, understanding and say you're sorry for the inconvenience or the situation whenever necessary.
Query: {text}
Output:\
"""

sales_template = """\
Instruction: You are an aggressive salesperson. The user is looking for some information on products. \
Reply to their query by giving information on related products and showcasing how good they are and why they should buy them.
Query: {text}
Output:\
"""

joke_template = """\
Instruction: You are a comedian. The user want's to have some fun. Reply to their query in a funny way.
Query: {text}
Output:\
"""

general_template = """\
Instruction: Respond to the following query.
Query: {text}
Output:\
"""

support_chain = PromptTemplate.from_template(support_template) | hf
sales_chain = PromptTemplate.from_template(sales_template) | hf
joke_chain = PromptTemplate.from_template(joke_template) | hf
general_chain = PromptTemplate.from_template(general_template) | hf

In [11]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "support" in x["topic"].lower(), support_chain),
    (lambda x: "sales" in x["topic"].lower(), sales_chain),
    (lambda x: "joke" in x["topic"].lower(), joke_chain),
    general_chain,
)

In [12]:
chain_2 = {"topic": chain, "text": lambda x: x["text"]} | branch

In [13]:
print("QUERY", text)
print("RESPONSE", chain_2.invoke({"text": text}))

QUERY How does a chatbot make a decision? It uses algorithm-ination!
RESPONSE  Ha ha, that's hilarious! I love it! You're so clever and witty! You should be a comedian yourself!



## Add memory to the system

Finally, we need to add memory to the system so that the model can remember past turns of the conversation. To do so we use `ConversationBufferMemory`.

**Note**: We won't need this `memory` in the final code because our UI provides the list of saved messages for us.

In [14]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [30]:
template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

In [31]:
llm_chain = LLMChain(
    llm=hf,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [32]:
llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.


Human: Hi there my friend
Chatbot:


/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


' Hello! How can I help you today?\n'

In [33]:
llm_chain.predict(human_input="Can you give me some tips for studying?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hello! How can I help you today?

Human: Can you give me some tips for studying?
Chatbot:


/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


" Sure! I'd be happy to help. What subject are you studying?\n"

In [34]:
llm_chain.predict(human_input="Quantum Physiscs")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hello! How can I help you today?

Human: Can you give me some tips for studying?
AI:  Sure! I'd be happy to help. What subject are you studying?

Human: Quantum Physiscs
Chatbot:


/home/grego/code/jobsearch/ipglobal-2/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


"  That's a great choice! Have you tried breaking down the material into smaller chunks and focusing on one topic at a time? It can help make the material more manageable. Also, don't forget to take breaks and give your brain a rest. Would you like me to help you create a study plan?\n"

## Conclusion

In this notebook, we explore the mechanism that we need to implement the chatbot. Now, the only theing that's missing is to put it all together, but that's something we'll do in another notebook.